In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pydeck as pdk
import folium
from folium.plugins import HeatMap

In [2]:
df = pd.read_csv('GTD.csv')

In [3]:
df = df[df['country_txt'] == 'Israel']
df=df[['latitude', 'longitude', 'nkill']].dropna()
df['nkill'] = df['nkill'].astype(int)
df['attack_indicator'] = 1 


In [4]:
print(df)

         latitude  longitude  nkill  attack_indicator
1014    31.771599  35.203400      1                 1
1314    32.004361  34.888229     26                 1
1498    31.771599  35.203400      0                 1
1499    31.771599  35.203400      0                 1
1500    31.771599  35.203400      0                 1
...           ...        ...    ...               ...
208201  31.670149  34.575154      0                 1
208772  33.015585  35.784354      0                 1
208854  31.670149  34.575154      0                 1
208984  31.927682  34.797671      0                 1
209568  31.670149  34.575154      0                 1

[2021 rows x 4 columns]


In [5]:
kill_frequencies = df['nkill'].value_counts().sort_index()
print(kill_frequencies)

nkill
0     1595
1      203
2       71
3       29
4       31
5       17
6       14
7        9
8        9
9        4
10       4
11       3
12       4
13       1
14       1
15       4
16       4
17       2
18       1
19       2
20       2
21       2
22       1
23       2
24       1
25       1
26       2
31       1
42       1
Name: count, dtype: int64


In [6]:
layer = pdk.Layer(
    "GridLayer",
    df,
    pickable=True,
    extruded=True, 
    cell_size=4000,  
    elevation_scale=90, 
    get_position=["longitude", "latitude"], 
    get_elevation="nkill",  
    elevation_range=[0, 3000], 
)

In [7]:
layer = pdk.Layer(
    "GridLayer",
    df,
    pickable=True,
    extruded=True, 
    cell_size=4000,  
    elevation_scale=90, 
    get_position=["longitude", "latitude"], 
    get_elevation="attack_indicator",  
    elevation_range=[0, 3000], 
)


In [8]:
layer = pdk.Layer(
    "HeatmapLayer",      
    df,                  
    get_position=['longitude', 'latitude'],
    get_weight= "nkill", 
    radius_pixels=60,    
    opacity=0.9          
)


In [9]:
layer = pdk.Layer(
    "HeatmapLayer",      
    df,                  
    get_position=['longitude', 'latitude'],
    get_weight= "attack_indicator", 
    radius_pixels=60,    
    opacity=0.9          
)


In [10]:
layer = pdk.Layer(
    "ScatterplotLayer", 
    df,
    get_position=["longitude", "latitude"],
    get_color=[255, 0, 0, 160],  
    get_radius=1000,  
    pickable=True
)


In [11]:
view_state = pdk.ViewState(
    latitude=31.0461, 
    longitude=34.8516,  
    zoom=7, 
    bearing=0,
    pitch=45
)
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
)
r.to_html("map.html")

In [12]:
attacks_per_location = df.groupby(['latitude', 'longitude']).size().reset_index(name='counts')

map_attacks = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=8)
HeatMap(data=attacks_per_location[['latitude', 'longitude', 'counts']].values.tolist(), radius=15).add_to(map_attacks)

map_attacks

In [13]:
kills_per_location = df.groupby(['latitude', 'longitude'])['nkill'].sum().reset_index()

map_kills = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=8)
HeatMap(data=kills_per_location[['latitude', 'longitude', 'nkill']].values.tolist(), radius=15).add_to(map_kills)

map_kills